In [2]:
import sys
# !{sys.executable} -m pip install xlrd
import xlrd
import os
import pandas as pd

In [4]:
# Set up directory structure however you want. Here the 'repaired' (as described above) .xls files are assumed to reside in the directory ./data/nvivo_content
NVIVO_CONTENT_FP = os.path.join(".") #, "data", "nvivo_content")
NOT_ERRORS_FP = os.path.join(NVIVO_CONTENT_FP, "everything_but_errors_12-22-20.xls")
ERRORS_FP = os.path.join(NVIVO_CONTENT_FP, "errors_12-22-20.xls")

everything_df = pd.read_excel(NOT_ERRORS_FP)
errors_df = pd.read_excel(ERRORS_FP)

In [7]:
everything_df = everything_df[["Hierarchical Name.1", "Coverage", "Number Of Coding References", "Reference Number", "Coded By Initials", "Coded Text"]]

In [10]:
errors_df = errors_df[["Hierarchical Name.1", "Coverage", "Number Of Coding References", "Reference Number", "Coded By Initials", "Coded Text"]]

In [47]:
# document-level (coverage)
document_level = everything_df[["Hierarchical Name.1", "Coverage", "Number Of Coding References"]]
document_level = document_level.drop_duplicates()
document_level


,Hierarchical Name.1,Coverage,Number Of Coding References
0,Files\\BWOH\\BWOH_Adair_Christia,0.0143,5
5,Files\\BWOH\\BWOH_Adams_Kathleen,0.0102,2
7,Files\\BWOH\\BWOH_Aiken_Harper_Rucker,0.0214,1
8,Files\\BWOH\\BWOH_Barnett_Etta,0.0237,2
10,Files\\BWOH\\BWOH_Duster_Alfreda,0.0027,2
...,...,...,...
4946,Files\\SCVF\\SCVF_Steinem_Gloria,0.0022,2
4948,Files\\SCVF\\SCVF_Toure_Nkenge,0.0564,5
4954,Files\\SHSA\\SHSA_Gartrell_Nanette,0.0328,2
4969,Files\\UNCTWMS\\UNCTWMS_Parsons_Darlene,0.1239,1


In [48]:
def extract_last_name(file_name):
    chunks = file_name.split("\\")
    # print(chunks)
    return chunks[4].split("_")[1]
def extract_first_name(file_name):
    chunks = file_name.split("\\")
    # print(chunks)
    return chunks[4].split("_")[2]
def extract_full_name(file_name):
    chunks = file_name.split("\\")
    # print(chunks)
    return chunks[4].split("_")[1] + ", " + chunks[4].split("_")[2]
def extract_collection(file_name):
    chunks = file_name.split("\\")
    return chunks[2]

document_level["first_name"] = document_level["Hierarchical Name.1"].apply(extract_first_name)
document_level["last_name"] = document_level["Hierarchical Name.1"].apply(extract_last_name)
document_level["collection_id"] = document_level["Hierarchical Name.1"].apply(extract_collection)
document_level["interviewee_name"] = document_level["Hierarchical Name.1"].apply(extract_full_name)


In [50]:
document_level = document_level[["collection_id", "interviewee_name", "Coverage", "Number Of Coding References"]]
document_level

,collection_id,interviewee_name,Coverage,Number Of Coding References
0,BWOH,"Adair, Christia",0.0143,5
5,BWOH,"Adams, Kathleen",0.0102,2
7,BWOH,"Aiken, Harper",0.0214,1
8,BWOH,"Barnett, Etta",0.0237,2
10,BWOH,"Duster, Alfreda",0.0027,2
...,...,...,...,...
4946,SCVF,"Steinem, Gloria",0.0022,2
4948,SCVF,"Toure, Nkenge",0.0564,5
4954,SHSA,"Gartrell, Nanette",0.0328,2
4969,UNCTWMS,"Parsons, Darlene",0.1239,1


In [69]:
# by collection
collection_comparison = document_level.groupby(['collection_id']).agg(['mean', 'count', 'max', 'min'])
collection_comparison

Coverage                        \
                          mean count     max     min   
collection_id                                          
BWOH                  0.008646    82  0.0418  0.0000   
BWSP                  0.011616    44  0.0605  0.0001   
HM                    0.011727   722  0.1369  0.0001   
Missing OSS and SOOH  0.007640    10  0.0359  0.0002   
OOHYLC                0.031500     2  0.0542  0.0088   
OSS                   0.012900     4  0.0295  0.0011   
OWHF                  0.036800     8  0.1606  0.0015   
ROHA                  0.007528    25  0.0483  0.0004   
RTRB                  0.013430    50  0.0693  0.0006   
SCAARJ                0.058509    32  0.2976  0.0016   
SCAL                  0.013340    20  0.0336  0.0008   
SCAP                  0.061396    24  0.3340  0.0006   
SCVF                  0.013913   155  0.1459  0.0001   
SHSA                  0.067165    17  0.2907  0.0019   
SHSF                  0.012245    22  0.0700  0.0001   
SOOH                  0.026700     6  0.0716  0.0005   
UNCGAS                0.056305    19  0.1954  0.0073   
UNCSW                 0.017284    32  0.0783  0.0002   
UNCTWMS               0.046636   188  0.4706  0.0006   
WOL                   0.028259    22  0.1454  0.0005   

                     Number Of Coding References                
                                            mean count max min  
collection_id                                                   
BWOH                                    1.914634    82   6   1  
BWSP                                    1.295455    44   5   1  
HM                                      2.560942   722  15   1  
Missing OSS and SOOH                    1.000000    10   1   1  
OOHYLC                                  1.000000     2   1   1  
OSS                                     1.000000     4   1   1  
OWHF                                    1.000000     8   1   1  
ROHA                                    1.160000    25   3   1  
RTRB                                    1.280000    50   4   1  
SCAARJ                                  5.062500    32  16   1  
SCAL                                    1.450000    20   3   1  
SCAP                                    1.333333    24   3   1  
SCVF                                    3.290323   155  24   1  
SHSA                                    4.941176    17  13   1  
SHSF                                    1.363636    22   3   1  
SOOH                                    1.000000     6   1   1  
UNCGAS                                  1.526316    19   4   1  
UNCSW                                   1.312500    32   4   1  
UNCTWMS                                 2.191489   188  12   1  
WOL                                     1.000000    22   1   1

In [66]:
# link to metadata
metadata = pd.read_csv("~/Downloads/Winter Quarter/OHTAP/OHTAP_metadata - Interviewees.csv")
metadata = metadata[(metadata['sex'] != "Male")]
metadata = metadata[["interviewee_name", "approximate_age_at_time_of_interview", "birth_decade", "collection_id"]]
metadata = metadata.drop_duplicates()
# check to drop audio interviews(?)
metadata

,interviewee_name,approximate_age_at_time_of_interview,birth_decade,collection_id
0,"Abbott, Jessie",80,1890,BWOH
1,"Adair, Christia",84,1890,BWOH
2,"Adams, Frankie V.",75,1900,BWOH
3,"Adams, Kathleen Redding",86,1890,BWOH
4,"Albrier, Frances Mary",81,1890,BWOH
...,...,...,...,...
5037,"Ebert, Chaz",65,1950,HM
5039,"Haddon, Phoebe",67,1950,HM
5040,"Ricketts, Vera",95,1920,HM
5043,"Rice, Constance",73,1940,HM


In [67]:
merged = pd.merge(metadata, document_level, on=["interviewee_name", "collection_id"]).drop_duplicates()
merged = merged[["interviewee_name", "approximate_age_at_time_of_interview", "birth_decade", "collection_id", "Coverage", "Number Of Coding References"]]
merged


,interviewee_name,approximate_age_at_time_of_interview,birth_decade,collection_id,Coverage,Number Of Coding References
0,"Adair, Christia",84,1890,BWOH,0.0143,5
1,"Adair, Christia",84,1890,BWOH,0.0038,2
2,"Adair, Christia",84,1890,BWOH,0.0041,1
3,"Adair, Christia",84,1890,BWOH,0.0143,3
4,"Adair, Christia",84,1890,BWOH,0.0143,4
...,...,...,...,...,...,...
1136,"Wesley, Valerie",70,1940,HM,0.0063,1
1137,"Wesley, Valerie",70,1940,HM,0.0006,1
1138,"Ebert, Chaz",65,1950,HM,0.0091,1
1139,"Ebert, Chaz",65,1950,HM,0.0018,1


In [71]:
# clear up overlapping ones
text_extraction = everything_df[["Hierarchical Name.1", "Number Of Coding References", "Reference Number", "Coded By Initials", "Coded Text"]]
def count_words(text):
    return len(text.split())
def has_overlap(text1, text2):
    text1
text_extraction["text_length"] = text_extraction["Coded Text"].apply(count_words)
text_extraction


,Hierarchical Name.1,Number Of Coding References,Reference Number,Coded By Initials,Coded Text,text_length
0,Files\\BWOH\\BWOH_Adair_Christia,5,1,NJMS,So I was the first volunteer secretary of the ...,207
1,Files\\BWOH\\BWOH_Adair_Christia,5,2,AA,So I was the first volunteer secretary of the ...,207
2,Files\\BWOH\\BWOH_Adair_Christia,5,3,NJMS,Knowing the role that you have played in many ...,239
3,Files\\BWOH\\BWOH_Adair_Christia,5,4,NN,"During the days I was with the NAACP, I almost...",121
4,Files\\BWOH\\BWOH_Adair_Christia,5,5,AA,"During the days I was with the NAACP, I almost...",136
...,...,...,...,...,...,...
4977,Files\\UNCTWMS\\UNCTWMS_Rostan_June,1,1,JL,"Well, you know, I think that because these wom...",96
4978,Files\\UNCTWMS\\UNCTWMS_Scott_Nan,3,1,JL,Always. JD: So what kind of instances? NS: Mos...,259
4979,Files\\UNCTWMS\\UNCTWMS_Scott_Nan,3,2,JL,So I started working in areas that were in the...,72
4980,Files\\UNCTWMS\\UNCTWMS_Scott_Nan,3,3,JL,I didn&#8217;t report him. JD: Could you ask ...,281


In [99]:
import difflib

def get_overlap(s1, s2):
    s = difflib.SequenceMatcher(None, s1, s2)
    pos_a, pos_b, size = s.find_longest_match(0, len(s1), 0, len(s2)) 
    return s1[pos_a:pos_a+size]

names = text_extraction["Hierarchical Name.1"].unique()
names
for name in names:
    relevant = text_extraction[text_extraction["Hierarchical Name.1"] == name]
    # print(relevant)
    texts = relevant["Coded Text"].unique().tolist()
    print(texts)
                

["So I was the first volunteer secretary of the branch here. Well, we had some pretty strong cases, cases like the DiMarco-Bob White case. Most of the cases were rape cases where white people used to become dissatisfied and disgruntled about a thing. The best way they could do, some white woman would holler that she'd been raped by a Negro or ventured to rape, or something like that, and of course that meant a burned or lynched Negro. And so this happened down at Conroe. But we were able to get a lawyer - this was one of NAACP's first projects - and a lawyer that won that case in our favor. And the day of the trial when he was declared innocent and turned loose, they shot him right in the jail house, I mean right in the courthouse down in Conroe, that's in Montgomery County. Well, those things were so exciting to me, I did not know that people could be treated like people in Houston were being treated. Not only in cases like that, but in jail, prisoners generally, why, they would beat 